In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.0 MB/s eta 0:00:00eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.5/388.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 18.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import asyncio
from openai import AsyncAzureOpenAI

client = AsyncAzureOpenAI(
    # This is the default and can be omitted
    api_key="ee2e2446ea81429db7d7b498ac4a8b09",
    api_version="2023-05-15",
    azure_endpoint="https://ai-loukritiaai771474864066.openai.azure.com/"
)


async def main(prompts) -> None:
    prompts =prompts[0]
    try:
        # Use the OpenAI API to create a chat completion
        chat_completion = await client.chat.completions.create(
            model="gpt-4",
            messages = prompts
        )
        
        # Print the response from the language model
        print(chat_completion.choices)
    except Exception as e:
        print(f"An error occurred: {e}")


In [4]:
async def main(prompts):
    try:
        # Use the OpenAI API to create a chat completion
        chat_completion = await client.chat.completions.create(
            model="gpt-4",
            messages=prompts
        )
        
        # Print the response from the language model
        for message in chat_completion.choices:
            print(message.message.content)
            return message.message.content
    except Exception as e:
        print(f"An error occurred: {e}")


In [5]:
import boto3
import pickle
from io import BytesIO

s3 = boto3.client('s3')

bucket_name = 'bucketnuke-469444003683-eu-central-1'  
folder_path = 'pickles_folder/' 

pickle_files = {
    'elife_test': 'elife_test_dataset.pkl',
    'elife_train': 'elife_train_dataset.pkl',
    'elife_val': 'elife_val_dataset.pkl',
    'plos_test': 'plos_test_dataset.pkl',
    'plos_train': 'plos_train_dataset.pkl',
    'plos_val': 'plos_val_dataset.pkl',
    'kids_train':'kids_train_dataset.pkl',
}

def load_dataset_from_s3(file_name):
    key = f"{folder_path}{file_name}"
    response = s3.get_object(Bucket=bucket_name, Key=key)
    content = response['Body'].read()
    dataset = pickle.loads(content)
    return dataset


In [6]:
plos_train_dataset = load_dataset_from_s3(pickle_files['plos_train'])

kids_train_dataset=load_dataset_from_s3(pickle_files['kids_train'])

In [7]:
elife_train_dataset = load_dataset_from_s3(pickle_files['elife_train'])

In [10]:
import random
import textwrap
from tqdm import tqdm

async def create_prompts_and_save(elife_dataset, kids_dataset):
    summaries_text = ""  

    for index, elife_item in tqdm(enumerate(elife_dataset['lay_summary'][:100])):
        example_prompts = [
                {
                    "role": "system",
                    "content": "Your goal is to enhance the accuracy and trustworthiness of a scientific summary intended for a broad audience. Concentrate on refining the facts, ensuring every statement is supported by the research, while maintaining clarity and accessibility. Aim to present the complexities of the study in a way that's both informative and understandable, eliminating any ambiguity or oversimplification."
                },
                
                {
                    "role": "user",
                    "content": elife_item
                },
            ]

        result = await main(example_prompts)
        summaries_text += f"{index + 1}:\n{elife_item}\n\n{result}\n\n"

    with open('summariesfactuality.txt', 'w') as file:
        file.write(summaries_text)


In [12]:
result = await create_prompts_and_save(elife_train_dataset, kids_train_dataset)

0it [00:00, ?it/s]

In the USA , more deaths happen in the winter than the summer . But when deaths occur varies greatly by sex , age , cause of death , and possibly region . Seasonal differences in death rates can change over time due to changes in factors that cause disease or affect treatment . Analyzing the seasonality of deaths can help scientists determine whether interventions to minimize deaths during a certain time of year are needed , or whether existing ones are effective . Scrutinizing seasonal patterns in death over time can also help scientists determine whether large-scale weather or climate changes are affecting the seasonality of death . Now , Parks et al . show that there are age and sex differences in which times of year most deaths occur . Parks et al . analyzed data on US deaths between 1980 and 2016 . While overall deaths in a year were highest in winter and lowest in summer , a greater number of young men died during summer – mainly due to injuries – than during winter . Seasonal di

1it [01:01, 61.47s/it]

The phenomenon of seasonal mortality, where the rate of deaths fluctuates with the seasons, is a complex subject of study in the United States. This variation is influenced by many factors, including age, sex, cause of death, and possibly geographic location. Over time, changes in disease prevalence, healthcare practices, and environmental factors can shift these patterns. Understanding these seasonal trends is pivotal for deploying targeted public health interventions aimed at reducing mortality during high-risk periods. Additionally, analyzing these trends over time can provide insights into how broader changes, such as climate shifts, might be influencing mortality.

Research conducted by Parks et al. has shed light on the intricate ways in which the seasonality of death varies across different demographics. By examining data on U.S. deaths from 1980 to 2016, their study revealed a consistent pattern: the highest overall mortality rates were observed in winter, whereas summer months

2it [01:33, 43.96s/it]

...

Malaria, a disease affecting hundreds of millions annually, is caused by parasites of the Plasmodium genus. Five different species of these parasites are known to infect humans, transmitted through the bites of Anopheles mosquitoes that carry the parasites. After transmission, the parasites quickly multiply within the human bloodstream. In areas endemic to malaria, repeated exposure can lead to individuals becoming infected yet remaining asymptomatic due to their immune system's capacity to suppress the parasite population to low levels. Furthermore, infection with multiple species of malaria parasites or the phenomenon of repeated infections is not uncommon.

One notable aspect of certain malaria species, such as Plasmodium vivax, is their ability to remain dormant in the liver for extended periods—weeks or even months—before reactivating. This dormancy and subsequent reactivation pose significant challenges in accurately determining the frequency of new malaria infections, as asympto

99it [2:22:09, 104.51s/it]

In every animal cell, just beneath the surface, lies a dynamic structure known as the cell cortex. This intricate network of filaments is not just a scaffold giving the cell its shape but holds a key role during cell division, particularly in a process called cytokinesis. During cytokinesis, the network reorganizes to form the cytokinetic ring, a crucial structure that contracts to bisect the cell, creating two new daughter cells. Guided by specific proteins and influenced by a phenomenon known as cortical flow—a mechanical movement of filaments toward the cell's center—the filaments arrange themselves into this ring. However, the extent to which cortical flow facilitates the formation of the cytokinetic ring remains an area of active investigation.

Caenorhabditis elegans, a small nematode worm, serves as a valuable model organism for studying the mechanisms of cell division due to the transparency and simplicity of its cells. In early C. elegans embryos, the stages of cell division c

100it [2:23:04, 85.84s/it]

Cells have developed intricate ways to adapt and respond to their environment by regulating the activity of genes. This process involves turning genes on or off, a task largely handled by a group of proteins known as transcription factors. These proteins influence the activity of specific, albeit overlapping, groups of genes. In essence, they collaborate in a manner akin to a dimmer switch, delicately adjusting the level of gene activity. This regulation occurs when the presence of transcription factors attracts other proteins, such as the Mediator complex. The Mediator complex plays a crucial role in gene activation, as it assembles the necessary protein machinery that reads and interprets the genes. Essentially, the concentration of transcription factors near a gene determines its activity level; more transcription factors lead to a stronger attraction of the Mediator, resulting in higher gene activity. A particular portion of the transcription factor, known as the activation domain,

In [30]:
# import textstat
# import numpy as np
# def calc_readability(pred):
#     print(textstat.flesch_kincaid_grade(pred))
#     print(textstat.coleman_liau_index(pred))
#     print(textstat.dale_chall_readability_score(pred))

In [14]:
import boto3
import pickle
from io import BytesIO

s3 = boto3.client('s3')

bucket_name = 'bucketnuke-469444003683-eu-central-1'  
folder_path = 'pickles_folder/' 

pickle_files = {
    'elife_test': 'elife_test_dataset.pkl',
    'elife_train': 'elife_train_dataset.pkl',
    'elife_val': 'elife_val_dataset.pkl',
    'plos_test': 'plos_test_dataset.pkl',
    'plos_train': 'plos_train_dataset.pkl',
    'plos_val': 'plos_val_dataset.pkl',
    'kids_train':'kids_train_dataset.pkl',
}

def load_dataset_from_s3(file_name):
    key = f"{folder_path}{file_name}"
    response = s3.get_object(Bucket=bucket_name, Key=key)
    content = response['Body'].read()
    dataset = pickle.loads(content)
    return dataset


In [15]:
elife_val_dataset = load_dataset_from_s3(pickle_files['elife_val'])
elife_train_dataset = load_dataset_from_s3(pickle_files['elife_train'])
elife_test_dataset = load_dataset_from_s3(pickle_files['elife_test'])

plos_val_dataset = load_dataset_from_s3(pickle_files['plos_val'])
plos_train_dataset = load_dataset_from_s3(pickle_files['plos_train'])
plos_test_dataset = load_dataset_from_s3(pickle_files['plos_test'])

kids_train_dataset=load_dataset_from_s3(pickle_files['kids_train'])

/home/ec2-user/envs/elifeplos/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
